In [1]:
%load_ext autoreload
%autoreload 2
from utils import *

In [2]:
df = pd.read_csv("csv/cmc.csv")
df.sample(5)

,wife's age,wife's education,husband's education,number of children,wife's religion,wife working,husband's occupation,standard-of-living index,media exposure,contraceptive method
831,29,2,3,5,1,1,2,4,0,3
1041,24,3,3,2,1,0,3,2,0,1
1126,31,4,4,2,1,1,1,2,0,1
643,27,2,2,5,1,1,3,1,0,3
930,37,3,4,4,0,1,2,4,0,3


In [6]:
print("wife's age : ", np.unique(df["wife's age"].values))
print("wife's education : ", np.unique(df["wife's education"].values))
print("husband's education : ", np.unique(df["husband's education"].values))
print("number of children : ", np.unique(df["number of children"].values))
print("wife's religion : ", np.unique(df["wife's religion"].values))
print("wife working : ", np.unique(df['wife working'].values))
print("husband's occupation : ", np.unique(df["husband's occupation"].values))
print("standard-of-living index : ", np.unique(df['standard-of-living index'].values))
print("media exposure : ", np.unique(df['media exposure'].values))
print("contraceptive method (class attribute) : ", np.unique(df['contraceptive method'].values))

wife's age :  [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45 46 47 48 49]
wife's education :  [1 2 3 4]
husband's education :  [1 2 3 4]
number of children :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 16]
wife's religion :  [0 1]
wife working :  [0 1]
husband's occupation :  [1 2 3 4]
standard-of-living index :  [1 2 3 4]
media exposure :  [0 1]
contraceptive method (class attribute) :  [1 2 3]


In [3]:
m = 9 # input dimension

In [7]:
vectors = df.values
dataset = LabeledSet(m)
dataset.addExamples(vectors[:,0:m], vectors[:,m:])

In [12]:
for i in range(3):
    print("number of instances of class " + str(i+1) + " : " + str(np.count_nonzero(dataset.y == i+1)))

number of instances of class 1 : 629
number of instances of class 2 : 333
number of instances of class 3 : 511


In [13]:
NMI1(dataset)

0.02587517341125771

In [25]:
nb_folds = 4

In [26]:
folds = get_folds(dataset, nb_folds)

In [15]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

# rank Gini discrimination measure 
g_g = One_minus()
rgdm = Gdm(h, g_g, f_r)

# Gini discrimination measure
gdm = Gdm(h, g_g, f)

# H_M
h_m = Max()
g_m = One_minus_square()
H_m = Gdm(h_m, g_m, f_r)

# H_Q
f_q = Avgdsr()
h_q = Square_root()
H_q = Gdm(h_q, g_g, f_q)

In [29]:
# Rank Shannon discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rsdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.4765707847295864
standard deviation of accuracy:  0.013366984364555547
average depth :  22.5
standard deviation of depth:  2.179449471770337
average number of leaves :  209.0
standard deviation of number of leaves:  8.74642784226795
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8132574061642156
standard deviation of ratio:  0.013171016154182142
average number of pairs used for ratio computing :  44.25
standard deviation of number of pairs:  2.5860201081971503


In [30]:
# Shannon entropy
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(sdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.5010217833156593
standard deviation of accuracy:  0.013767299630903314
average depth :  17.5
standard deviation of depth:  0.8660254037844386
average number of leaves :  163.75
standard deviation of number of leaves:  2.277608394786075
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.805322514798418
standard deviation of ratio:  0.03435368666326917
average number of pairs used for ratio computing :  46.0
standard deviation of number of pairs:  2.449489742783178


In [31]:
# Rank Gini discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rgdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.4772501325556734
standard deviation of accuracy:  0.02018850403337266
average depth :  22.0
standard deviation of depth:  2.5495097567963922
average number of leaves :  206.25
standard deviation of number of leaves:  14.549484526951462
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8055406041783121
standard deviation of ratio:  0.009552454575138392
average number of pairs used for ratio computing :  44.75
standard deviation of number of pairs:  3.2691742076555053


In [32]:
# Gini entropy
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(gdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.5010236243666785
standard deviation of accuracy:  0.013563061718679957
average depth :  18.0
standard deviation of depth:  1.5811388300841898
average number of leaves :  168.25
standard deviation of number of leaves:  2.7726341266023544
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8172683007825272
standard deviation of ratio:  0.021368805428500406
average number of pairs used for ratio computing :  42.0
standard deviation of number of pairs:  1.224744871391589


In [33]:
# Pessimistic rank discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(prdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.4827051667255803
standard deviation of accuracy:  0.022528232016500328
average depth :  27.5
standard deviation of depth:  1.8027756377319946
average number of leaves :  204.25
standard deviation of number of leaves:  3.2691742076555053
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8364652944827976
standard deviation of ratio:  0.016113283325322567
average number of pairs used for ratio computing :  40.25
standard deviation of number of pairs:  2.8613807855648994


In [34]:
# H_M
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_m, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.49220314893366324
standard deviation of accuracy:  0.028627196723849585
average depth :  31.25
standard deviation of depth:  1.920286436967152
average number of leaves :  223.25
standard deviation of number of leaves:  5.717298313014636
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8534047782031717
standard deviation of ratio:  0.019976113900407943
average number of pairs used for ratio computing :  36.0
standard deviation of number of pairs:  2.7386127875258306


In [36]:
# H_Q
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_q, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2, 3])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.4602848474136915
standard deviation of accuracy:  0.011584701736204904
average depth :  21.5
standard deviation of depth:  2.29128784747792
average number of leaves :  207.0
standard deviation of number of leaves:  6.4031242374328485
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.803776467949262
standard deviation of ratio:  0.02086577346058918
average number of pairs used for ratio computing :  47.0
standard deviation of number of pairs:  3.082207001484488
